In [1]:


import torch

from torch.utils.data import Dataset, DataLoader



from torchvision import transforms as T, utils

import matplotlib.pyplot as plt
from PIL import Image





#ab diffusion related imports
from AB_diffusion.user_hints import RandomHintGenerator
from AB_diffusion.color_handling import de_normalize_lab, normalize_lab,plotMinMax
from AB_diffusion.ab_denoising_diffusion_pytorch import ABUnet, ABGaussianDiffusion
from AB_diffusion.ab_trainer import ABDataset
from AB_diffusion.colorizer_app import ColorizerApp
from IPython.utils import io as iol
from skimage import io
from kornia.color import rgb_to_lab, lab_to_rgb
from datasets import load_from_disk,load_dataset
#from ab_classifier_free_guidance import *


import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import random
import glob
import mplcursors
import ipywidgets as widgets
from IPython.display import display
import torchvision.transforms as transforms

from PIL import Image
#import ipywidgets as widgets
#from IPython.display import display, clear_output
#from PIL import Image
#import io


In [2]:
device = torch.device(1 if torch.cuda.is_available() else "cpu")
print(device)
print("Selected GPU:", torch.cuda.get_device_name(device))
#check wicj gpu is selected
torch.cuda.set_device(device)

cuda:1
Selected GPU: A100-SXM4-80GB


In [3]:
from ema_pytorch import EMA


def load(model_name,folder):
    print(str(folder  + f'/{model_name}'))
    data = torch.load(str(folder  + f'/{model_name}'), map_location=device)
    return data

model = ABUnet(
    dim = 64,
    out_dim = 2,
    channels=5,
    full_attn = (False, False, False, True)    

).to(device)
diffusion = ABGaussianDiffusion(
    model,
        image_size = 256,
        timesteps = 1000,
        objective = 'pred_v',
        beta_schedule = 'sigmoid',
        min_snr_loss_weight = True, # https://arxiv.org/abs/2303.09556
        min_snr_gamma = 5,
        offset_noise_strength=0.1
).to(device)
with iol.capture_output() as captured:

    model_folder = "./results/fs_imgnet-pred_v-1000-sigmoid-256"
    model_name = "model-pred_v-1000-sigmoid-256-10x10000steps.pt"
    #model_folder = "./results/no_reinit-pred_v-1000-sigmoid-256"
    #model_name = "model-pred_v-1000-sigmoid-256-121x1000steps.pt"
    loaded_data = load(model_name,model_folder)
    #diffusion_model.load_state_dict(loaded_data['model'])
    #diffusion_model.to(device)
    ema = EMA(diffusion)
    ema.load_state_dict(loaded_data['ema'])
    ema.to(device)

timesteps 1000


In [4]:
path = "./experiments/comp/tennis.jfif"

In [5]:
colorizer = ColorizerApp(path, ema,256, device)
colorizer.run()
